In [ ]:
WORKING_DIR = u"/path/to/your/folder/contains/music"  # no '/' as end is ok
FILENAME_PREFIX = u"your_music_file_name_without_ext"

OUTPUT_PREFIX = u"/path/to/your/music/output/files"  # ends with '/'

PICTURE = u"Front.jpg"
ANSI_ENCODING = "gbk"

FILES_TO_COPY = ["Booklet.zip"]

DELETE_TARGET_DIR = False  # If clean the target folder at first

EXTRA_DATA_FILE = u"%s.ini" % FILENAME_PREFIX
INPUT_FILE = u"%s.flac" % FILENAME_PREFIX
ANSI_CUE = u"%s.cue" % FILENAME_PREFIX
UTF8_CUE = u"%s.utf8.cue" % FILENAME_PREFIX

In [ ]:
import codecs
from collections import defaultdict
import re
import subprocess
import ConfigParser
import os, sys

In [ ]:
sys.path.append("/path/to/your/repository/of/GatesMusicPet/")
from music_pet import utils
from music_pet import meta

In [ ]:
global_report = []

NOT_PARSED = 1
NO_TRACK = 2

In [ ]:
def utf8(data):
    if type(data) == str:
        return codecs.decode(data, "utf8")
    elif type(data) == unicode:
        return data
    else:
        return codecs.decode(str(data), "utf8")


### Convert CUE to utf8

In [ ]:
cd $WORKING_DIR

In [ ]:
try:
    utils.remove_bom(ANSI_CUE, UTF8_CUE)
except ValueError as ex:
    print(ex)

In [ ]:
if not os.path.exists(UTF8_CUE):
    with open(ANSI_CUE, "r") as fp:
        ansi_content = fp.read()
        with open(UTF8_CUE, "w") as fp2:
            fp2.write(codecs.encode(
                codecs.decode(ansi_content, ANSI_ENCODING),
                "utf-8"))

### Parse CUE

In [ ]:
cue = meta.CUE(UTF8_CUE)

In [ ]:
try:
    extra_info = meta.ExtraInfo(EXTRA_DATA_FILE)
    extra_info.update_meta(cue.meta)
except Exception as ex:
    print(ex)

In [ ]:
print(cue.details())

### Covert Files

In [ ]:
cd $WORKING_DIR

In [ ]:
rm __*.wav

In [ ]:
!ffmpeg -y -i "$INPUT_FILE" "__temp.wav"

In [ ]:
!mkdir "$OUTPUT_PREFIX"

In [ ]:
def filename_safe(filename):
    for ch in '<>"?*\\/:':
        filename = filename.replace(ch, "_")
    return filename.strip()

In [ ]:
def filename_format(meta, track_num):
    if u"title" not in meta[unicode(track_num)]:
        raise ValueError

    filename = meta[unicode(track_num)][u"title"]
    if u"tracknumber" in meta[unicode(track_num)]:
        filename = u"%s %s" % (meta[unicode(track_num)][u"tracknumber"].zfill(2), filename)
    if u"discnumber" in meta[u"0"]:
        filename = u"%s-%s" % (meta[u"0"][u"discnumber"], filename)
    return filename

In [ ]:
folder = u"%s%s" % (OUTPUT_PREFIX, filename_safe("%s" % utf8(cue.meta[u"0"]["album"])))
if DELETE_TARGET_DIR:
    !rm -r "$folder"
    pass
!mkdir -p "$folder"

if PICTURE is not None:
    !cp "$PICTURE" "$folder"

In [ ]:
def cue_index_to_flac_time(text):
    r = re.search(u'''(\d+:\d{2}):(\d{2})''', text)
    if r:
        return u"%s.%s" % (r.groups()[0], r.groups()[1])
    return None

In [ ]:
for track_num in xrange(1, 1 + int(cue.meta[u"0"][u"tracktotal"])):
    cmd = [
        u"flac",
        u"--best"
    ]
    
    # Calculate time
    if track_num > 1:
        cmd.append(u'--skip=%s' % cue_index_to_flac_time(cue.meta[unicode(track_num)]["index_01"]))
    if track_num < int(cue.meta[u"0"][u"tracktotal"]):
        cmd.append(u'--until=%s' % cue_index_to_flac_time(cue.meta[unicode(track_num + 1)]["index_01"]))
    
    if PICTURE is not None:
        cmd.append(u'--picture="%s"' % utils.to_unicode(PICTURE))
    
    # appending tags
    for key, value in cue.meta[unicode(track_num)].items():
        cmd.append(u'--tag="%s"="%s"' % (key, value))
    for key, value in cue.meta[u"0"].items():
        if key not in cue.meta[unicode(track_num)]:
            cmd.append(u'--tag="%s"="%s"' % (key, value))
    
    filename = filename_safe("%s.flac" % filename_format(cue.meta, track_num))
    cmd.append(u'-o "%s/%s"' % (folder, filename))
    cmd.append(u'"__temp.wav"')
    
    print("===Encoding Track %d===" % track_num)
    
    cmdline = " ".join(cmd)
    print(cmdline)
    print(subprocess.check_output(cmdline,
                                  shell=True,
                                  stderr=subprocess.STDOUT))

In [ ]:
for f in FILES_TO_COPY:
    !cp "$f" "$folder"

In [ ]:
!rm __*.wav

### *playground*